# HMM

Hidden Markov Model, HMM 是可用于标注问题的统计学习模型，属于**生成模型**。由**初始概率分布**，**状态转移概率分布**和**观测概率分布**确定。
$\lambda = (A, B, \pi)$.  


### **两个基本假设：**  
1). 齐次马尔可夫性假设，即假设隐藏的马尔可夫链在任意时刻$t$的状态只依赖于其前一时刻的状态，与其他时刻状态及观测无关，也与时刻$t$无关：  
$P(i_{t}|i_{t-1},o_{t-1},...,i_{1},o_{1}) = P(i_{t}|i_{t-1}),  t = 1,2,...,T$  

2）. 观察独立性假设，即假设任意时刻的观测只依赖与该时刻的马尔可夫链的状态，与其他观测及状态无关：  
$P(o_{t}|i_{T},o_{T},i_{T-1},o_{T-1},...,i_{t+1},o_{t+1},i_{t},i_{t-1},o_{t-1},...,i_{1},o_{1} = P(o_{t}|i_{t})$


### **三个基本问题**:   

1. **概率计算问题**。给定模型$\lambda = (A, B, \pi)$ 和观测序列 $O=(o_{1},o_{2},...,o_{T})$, 计算在模型 $\lambda$ 下观测序列 $O$ 出现的概率 $P(O|\lambda)$.  
Evaluate $P(O|\lambda)$.

2. **学习问题**。已知观测序列 $O=(o_{1},o_{2},...,o_{T})$, 估计模型 $\lambda = (A, B, \pi)$ 的参数，使得在该模型下观测序列概率 $P(O|\lambda)$ 最大。即用极大似然估计的方法估计参数。  
$\lambda_{MLE} = argmax_{\lambda}P(O|\lambda)$.

3. **预测问题**。也称为解码(decoding) 问题。已知模型 $\lambda = (A, B, \pi)$ 和观测序列 $O=(o_{1},o_{2},...,o_{T})$，求给定观测序列条件概率 $P(I|O)$ 最大的状态序列 $I = (i_{1}, i_{2}, i_{3},...,i_{T})$. 即给定观测序列，求最有可能的对应的状态序列。  
$argmax_{I}P(I|O,\lambda)$


问题1. 前向（forward）和后向（backward）算法。  
问题2. Baum-Welch 算法。  
问题3. 近似算法，维特比算法。 

---------------------------------------------------------------------------------------------------------------------------------

以下来自徐亦达的课程  
视频地址：https://www.youtube.com/watch?v=Ji6KbkyNmk8  
lecture： https://github.com/roboticcam/machine-learning-notes/blob/master/dynamic_model.pdf

# 概率计算问题

---------------------------------------------------------------------------------------------------------------------------------

### 例 10.2  


考虑盒子和球模型$\lambda = (A, B, \pi)$ , 状态集合 $Q = {1,2,3}$, 观测集合 $V = {红,白}$,  


$A=\begin{bmatrix}
 0.5&  0.2& 0.3\\ 
 0.3&  0.5& 0.2\\ 
 0.2&  0.3& 0.5
\end{bmatrix}, 
B=\begin{bmatrix}
 0.5&  0.5\\ 
 0.4&  0.6\\ 
 0.7&  0.3
\end{bmatrix},
\pi=\begin{bmatrix}
 0.2\\ 
 0.4\\ 
 0.4
\end{bmatrix}$  

设$T=3, Q={红,白,红}$,试用前向算法计算$P(O|\lambda)$.  





In [0]:
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2],[0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
pi = [0.2, 0.4, 0.4]
Q = [0,1,0]

In [0]:
import numpy as np

In [0]:
class HMM_fw:
    def __init__(self, A, B, pi):
        self.A = A    # 状态转移概率
        self.B = B    # 观测概率
        self.pi = pi  # 初始状态
        
    def forward(self, Q):
        T = len(Q)   # 观测序列长度，时刻T
        M = len(self.A)   # 状态数
        alpha = np.zeros((T, M))
        
        for t in range(T):
            for m in range(M):
                if t == 0:
                    alpha[t][m] = self.pi[m] * self.B[m][Q[t]]
                    print("alpha[{}][{}] = pi[{}] * B[{}](Q{}) = {:.2f}".format(t+1, m+1, m+1, m+1, Q[t]+1, alpha[t][m]))
                else:
                    alpha[t][m] = sum([alpha[t-1][i] * self.A[i][m] for i in range(len(alpha[t-1]))]) * self.B[m][Q[t]]
                    print("alpha[{}][{}] = {:.5f}".format(t+1, m+1, alpha[t][m]))
                    
        p = sum(alpha[T-1])
        #print(p)
        return p

In [79]:
m = HMM_fw(A, B, pi)
m.forward(Q)

alpha[1][1] = pi[1] * B[1](Q1) = 0.10
alpha[1][2] = pi[2] * B[2](Q1) = 0.16
alpha[1][3] = pi[3] * B[3](Q1) = 0.28
alpha[2][1] = 0.07700
alpha[2][2] = 0.11040
alpha[2][3] = 0.06060
alpha[3][1] = 0.04187
alpha[3][2] = 0.03551
alpha[3][3] = 0.05284


0.130218

In [0]:
class HMM_bw:
    def __init__(self, A, B, pi):
        self.A = A
        self.B = B
        self.pi = pi
        
    def backward(self, Q):
        T = len(Q)   # 观测序列长度，时刻T
        N = len(self.A)   # 状态数
        beta = np.zeros((T, N))
        
        for t in range(T-1, -1, -1):
            for n in range(N):
                if t == T - 1:
                    beta[t][n] = 1
                    print("beta[{}][{}] = {:.2f}".format(t+1, n, beta[t][n]))
                else:
                    beta[t][n] = sum(self.A[n][j] * self.B[j][Q[t+1]] * beta[t+1][j] for j in range(N))
                    print("beta[{}][{}] = {:.5f}".format(t+1, n, beta[t][n]))
                    
        p = sum(self.pi[i] * self.B[i][Q[0]] * beta[0][i] for i in range(N))
        #print(p)
        return p

In [77]:
m = HMM_bw(A, B, pi)
m.backward(Q)

beta[3][0] = 1.00
beta[3][1] = 1.00
beta[3][2] = 1.00
beta[2][0] = 0.54000
beta[2][1] = 0.49000
beta[2][2] = 0.57000
beta[1][0] = 0.24510
beta[1][1] = 0.26220
beta[1][2] = 0.22770


0.130218

# 学习问题

HMM的学习，根据训练数据是否包括观测序列和对应的状态序列还是只有观测序列，可以分别为**有监督学习**和**无监督学习**来实现。

### 监督学习  

假设已给训练数据包含$S$个长度相同的观测序列和对应的状态序列${(O_{1}, I_{1}), (O_{2}, I_{2}),..., (O_{S}, I_{S})}$. 那么可以利用**极大似然估计**法来估计HMM的参数。

### 无监督学习  

假设已给训练数据只包含$S$个长度为$T$的观测序列${O_{1}, O_{2},..., O_{S}}$, 而没有对应的状态序列， 目标是学习HMM $\lambda = (A, B, \pi)$ 的参数。 我们将观测序列数据看作观测数据$Q$, 状态序列数据看作不可观测的隐数据$I$, 那么HMM则是一个含有隐变量的概率模型：  

$P(O|\lambda) = \sum_{I}P(O|I, \lambda)P(I|\lambda)$  

他的参数可以由EM算法来学习。

# 预测问题


### 近似算法  
近似算法的想法是， 在每个时刻$t$ 选择在该时刻最有可能出现的状态$i^{*}_{t}$，从而得到一个状态序列 $I^{*} = (i^{*}_{1}, i^{*}_{2}, ..., i^{*}_{T})$, 将他作为预测的结果。

### 维特比算法  


维特比算法实际是用动态规划，解HMM的预测问题，即用动态规划求概率最大路径。

In [0]:
class HMM_viterbi:
    def __init__(self, A, B, pi):
        self.A = A
        self.B = B
        self.pi = pi
        
    def viterbi(self, Q):
        T = len(Q)   # 观测序列长度，时刻T
        N = len(self.A)   # 状态数
        sigma = np.zeros((T, N))
        delta = np.zeros((T, N))
        for t in range(T):
            for n in range(N):
                if t == 0:
                    sigma[t][n] = self.pi[n] * self.B[n][Q[t]]
                    delta[t][n] = 0
                    print("sigmia[{}][{}] = {:.2f}".format(t+1, n+1, sigma[t][n]))
                    print("delta[{}][{}] = {}".format(t+1, n+1, delta[t][n]))
                    
                else:
                    sigma[t][n] = np.max([sigma[t-1][j] * self.A[j][n] for j in range(N)]) * self.B[n][Q[t]]
                    print("sigma[{}][{}] = {:.5f}".format(t+1, n+1, sigma[t][n]))
                    
                    delta[t][n] = np.argmax([sigma[t-1][j] * self.A[j][n] for j in range(N)]) + 1
                    print("delta[{}][{}] = {}".format(t+1, n+1, delta[t][n]))
                    
        P = np.max(sigma[T-1])
        print(P)
        pth = []
        for t in range(T-1, -1, -1):
            if t == T - 1:
                i_t = np.argmax(sigma[t])
                pth.append(i_t + 1)
            else:
                i_t = int(delta[t+1][i_t]) - 1
                pth.append(i_t + 1)
        
        return pth

#### 例 10.3

$A=\begin{bmatrix}
 0.5&  0.2& 0.3\\ 
 0.3&  0.5& 0.2\\ 
 0.2&  0.3& 0.5
\end{bmatrix}, 
B=\begin{bmatrix}
 0.5&  0.5\\ 
 0.4&  0.6\\ 
 0.7&  0.3
\end{bmatrix},
\pi=\begin{bmatrix}
 0.2\\ 
 0.4\\ 
 0.4
\end{bmatrix}$  

已知观测序列$O=(红, 白, 红)$,试求最优状态序列，即最优路径 $I^{*}=(i^{*}_{1}, i^{*}_{2}, i^{*}_{3})$.

In [0]:
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2],[0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
pi = [0.2, 0.4, 0.4]
Q = [0,1,0]

In [135]:
m = HMM_viterbi(A, B, pi)
m.viterbi(Q)

sigmia[1][1] = 0.10
delta[1][1] = 0.0
sigmia[1][2] = 0.16
delta[1][2] = 0.0
sigmia[1][3] = 0.28
delta[1][3] = 0.0
sigma[2][1] = 0.02800
delta[2][1] = 3.0
sigma[2][2] = 0.05040
delta[2][2] = 3.0
sigma[2][3] = 0.04200
delta[2][3] = 3.0
sigma[3][1] = 0.00756
delta[3][1] = 2.0
sigma[3][2] = 0.01008
delta[3][2] = 2.0
sigma[3][3] = 0.01470
delta[3][3] = 3.0
0.014699999999999998


[3, 3, 3]